# Оптимизация выполнения кода, векторизация, Numba

## Задачи для совместного разбора

In [129]:
# !pip install git+https://github.com/pyutils/line_profiler.git

In [130]:
!pip install line_profiler

In [131]:
import numpy 
numpy.__version__

'1.20.1'

In [132]:
import numpy as np
from numba import njit

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [133]:
A = np.random.randint(0, 1000, size=(1000000))
A

array([476, 176, 176, ..., 855, 497, 744])

In [134]:
def f1(A):
    acc = 0
    cnt = 0
    for i in range(len(A)):
        bi = A[i] + 100
        acc += bi
        cnt += 1
    return acc / cnt

In [135]:
%%time
f1(A)

Wall time: 689 ms


599.98505

In [136]:
%time f1(A)

Wall time: 838 ms


599.98505

In [137]:
%%timeit
f1(A)

714 ms ± 39.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [138]:
def f2(A):
    acc = 0
    n = len(A)
    for i in range(n):
        acc += A[i]
 
    return acc / n + 100

In [139]:
%%time
f2(A)

Wall time: 369 ms


599.98505

In [140]:
%%time
A.mean() + 100

Wall time: 15.6 ms


599.98505

In [141]:
import numba

@njit
def f3(A):
    acc = 0
    n = len(A)
    for i in range(n):
        acc += A[i]
    return acc / n + 100

In [142]:
%%time
f3(A)

Wall time: 1.18 s


599.98505

In [143]:
%%time
f3(A)

Wall time: 0 ns


599.98505

2. Напишите функцию, которая возвращает сумму всех чисел от 0 до x-1. Примените функцию к каждому элементу массива.

In [144]:
def g(x):
    return sum(range(x))

In [145]:
%%time
np.array([g(x) for x in A])

Wall time: 16.3 s


array([113050,  15400,  15400, ..., 365085, 123256, 276396])

In [146]:
g_v = np.vectorize(g)

In [147]:
%%time
g_v(A)

Wall time: 15.8 s


array([113050,  15400,  15400, ..., 365085, 123256, 276396])

In [148]:
A.dtype

dtype('int32')

In [149]:
numba.__version__

'0.56.3'

In [150]:
# !pip3 install --user --upgrade numba==0.56.3 нужно было обновить

In [151]:
g_v2 = numba.vectorize(["int32(int32)"])(g)

In [152]:
%%time
g_v2(A)

Wall time: 144 ms


array([113050,  15400,  15400, ..., 365085, 123256, 276396])

3. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [153]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [154]:
import pandas as pd
import string

N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

,col0,col1,col2,col3,key
0,-0.201036,0.566161,0.930465,-0.915176,v
1,-0.270020,0.506523,1.695031,0.842120,h


In [155]:
def h1(df):
    mask = []
    for _, row in df.iterrows():
        mask.append(row["key"] in {"a", "b", "c", "d", "e"})
    return df[mask]

In [156]:
%%time
h1(df.head(50000))

Wall time: 6.31 s


,col0,col1,col2,col3,key
11,-0.521541,-0.385167,0.561132,0.573517,c
14,-0.052285,-1.651434,1.149881,-1.316376,e
18,0.367222,1.664914,0.745809,0.221520,e
25,0.164771,-0.626609,-0.354515,0.542849,c
38,-0.140512,-2.327106,0.080031,2.087057,b
...,...,...,...,...,...
49973,-0.734825,0.293530,-0.977299,-0.356086,e
49976,-0.620289,1.151120,-0.333062,1.417203,e
49989,-0.481288,-0.026819,0.328508,0.155043,d
49990,-1.424480,-0.303901,-1.274494,1.023782,b


In [157]:
%lprun -f h1 h1(df.head(50000))

In [158]:
%%time
df[df["key"].isin({"a", "b", "c", "d", "e"})]

Wall time: 187 ms


,col0,col1,col2,col3,key
11,-0.521541,-0.385167,0.561132,0.573517,c
14,-0.052285,-1.651434,1.149881,-1.316376,e
18,0.367222,1.664914,0.745809,0.221520,e
25,0.164771,-0.626609,-0.354515,0.542849,c
38,-0.140512,-2.327106,0.080031,2.087057,b
...,...,...,...,...,...
1999954,-0.325398,-0.754458,0.351753,0.013413,a
1999959,1.715718,-0.315842,-0.325200,0.435970,c
1999968,1.052491,0.192632,-1.092525,-0.394541,b
1999971,-0.453339,0.515297,-2.467061,-0.778647,d


## Лабораторная работа 3

In [1]:
!pip3 install --user --upgrade numba==0.56.3

In [2]:
!pip install line_profiler

In [3]:
!pip install iteration_utilities

In [4]:
import re
import nltk
import math
import json
import numba
import numpy as np
import pandas as pd
from numba import prange
from typing import Union
from numba import jit, njit
from collections import Counter
from iteration_utilities import flatten

%load_ext line_profiler

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sanha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

In [6]:
recipes = pd.read_csv("recipes_sample.csv")
recipes["submitted"] = pd.to_datetime(recipes['submitted'], format="%Y-%m-%d")
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [7]:
recipes.dtypes

name                      object
id                         int64
minutes                    int64
contributor_id             int64
submitted         datetime64[ns]
n_steps                  float64
description               object
n_ingredients            float64
dtype: object

In [8]:
reviews = pd.read_csv("reviews_sample.csv", index_col = 0)
reviews["date"] = pd.to_datetime(reviews['date'], format="%Y-%m-%d")
reviews.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [9]:
reviews.dtypes

user_id               int64
recipe_id             int64
date         datetime64[ns]
rating                int64
review               object
dtype: object

## Измерение времени выполнения кода

Назовем полным описанием рецепта строку, полученную путем конкатенации названия и описания рецепта через пробел. Удалите строки для рецептов, которые были добавлены не в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году.

In [10]:
recipes = recipes[recipes['submitted'].dt.year == 2010]

1\.1 С использованием метода `DataFrame.iterrows` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - нахождение полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [11]:
%%time

def get_mean_len_A(df: pd.DataFrame) -> float:
    amount_of_words = 0
    for index, row in df.iterrows():
        amount_of_words += len(row["name"] + " " + row["description"])
    return amount_of_words/len(df)
                               
res1 = get_mean_len_A(recipes)
print(res1)

265.501300390117
Wall time: 90 ms


In [12]:
%%timeit

def get_mean_len_A(df: pd.DataFrame) -> float:
    amount_of_words = 0
    for index, row in df.iterrows():
        amount_of_words += len(row["name"] + " " + row["description"])
    return amount_of_words/len(df)
                               
get_mean_len_A(recipes)

85.2 ms ± 326 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


1\.2. С использованием метода `DataFrame.apply` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - вызываете метод apply у таблицы, в качестве аргумента передаете функцию, которая возвращает полное описание для каждой строки;
    
    - считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [13]:
%%time

def get_mean_len_B(df: pd.DataFrame) -> float:
    return df[['name','description']].apply(lambda x : '{} {}'.format(x[0],x[1]), axis=1).str.len().mean()

res2 = get_mean_len_B(recipes)
print(res2)

265.501300390117
Wall time: 13 ms


In [14]:
%%timeit

def get_mean_len_B(df: pd.DataFrame) -> float:
    return df[['name','description']].apply(lambda x : '{} {}'.format(x[0],x[1]), axis=1).str.len().mean()

get_mean_len_B(recipes)

11.1 ms ± 39.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


1\.3. С использованием векторизованных методов серий `pd.Series`:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - при помощи векторизированных операций получаете столбец с полным описанием;
    
    - при помощи векторизированных операций получаете длины полного описания;
    
    - при помощи векторизированных операций получаете среднюю длину полных описаний. 

In [15]:
%%time

def get_mean_len_C(df: pd.DataFrame) -> float:
    return df.name.str.cat(df.description, sep=' ').str.len().mean()

res3 = get_mean_len_C(recipes)
print(res3)

265.501300390117
Wall time: 2 ms


In [16]:
%%timeit

def get_mean_len_C(df: pd.DataFrame) -> float:
    return df.name.str.cat(df.description, sep=' ').str.len().mean()

get_mean_len_C(recipes)

1.17 ms ± 7.14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [17]:
if res1 == res2 == res3: print("результаты работы всех написанных функций корректны и совпадают.")

результаты работы всех написанных функций корректны и совпадают.


## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. 

In [18]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        recipe_id, review = row["recipe_id"], row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально. 

In [19]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-07 s

Total time: 51.7825 s
File: <ipython-input-184-21baa068d50d>
Function: get_word_reviews_count at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4                                           def get_word_reviews_count(df):
     5         1         24.0     24.0      0.0      word_reviews = {}
     6    126680  157647941.0   1244.5     30.4      for _, row in df.dropna(subset=["review"]).iterrows():
     7    126679   21338263.0    168.4      4.1          recipe_id, review = row["recipe_id"], row["review"]
     8    126679   13862539.0    109.4      2.7          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
     9   6918689   20155386.0      2.9      3.9          for word in words:
    10   6792010   28820539.0      4.2      5.6              if word not in word_reviews:
    11     93059     442493.0      4.8      0.1                  word_reviews[word] = []
    12   6792010   32216516.0      4.7      6.2              word_reviews[word].append(recipe_id)
    13                                           
    14         1         27.0     27.0      0.0      word_reviews_count = {}
    15    126680  153098317.0   1208.5     29.6      for _, row in df.dropna(subset=["review"]).iterrows():
    16    126679   12208275.0     96.4      2.4          review = row["review"]
    17    126679   13889712.0    109.6      2.7          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
    18   6918689   20611264.0      3.0      4.0          for word in words:
    19   6792010   43533803.0      6.4      8.4              word_reviews_count[word] = len(word_reviews[word])
    20         1         16.0     16.0      0.0      return word_reviews_count

Главной проблемой в данном коде является использование iterrows. Мы видим что в сумме только на iterrows потрачено 60% всего времени. Кроме того итерация по словам и работа функции append так же занимает немалое количество времени.

2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

#### Мой вариант

In [20]:
def faster_get_word_reviews_count(df):
    word_counts = Counter(list(flatten(df.review.apply(lambda x: re.sub("[^A-Za-z\s]", "", str(x)).split(" ")))))
    return word_counts.most_common()

In [21]:
%%time

faster_get_word_reviews_count(reviews)

Wall time: 2.13 s


[('', 330966),
 ('I', 288243),
 ('the', 266530),
 ('and', 219215),
 ('a', 166467),
 ('it', 133219),
 ('to', 132170),
 ('for', 123162),
 ('of', 109239),
 ('this', 93873),
 ('was', 89617),
 ('recipe', 71751),
 ('in', 62796),
 ('with', 59275),
 ('is', 56519),
 ('so', 46848),
 ('used', 45538),
 ('my', 44644),
 ('but', 43035),
 ('This', 39555),
 ('made', 39363),
 ('that', 38949),
 ('make', 37321),
 ('on', 36121),
 ('as', 35863),
 ('have', 35532),
 ('Thanks', 32325),
 ('good', 31181),
 ('you', 28143),
 ('had', 28047),
 ('great', 27326),
 ('very', 26040),
 ('The', 25845),
 ('them', 25768),
 ('out', 25697),
 ('be', 25514),
 ('just', 25271),
 ('time', 24744),
 ('will', 24097),
 ('not', 23840),
 ('added', 22099),
 ('easy', 21683),
 ('were', 21189),
 ('again', 21144),
 ('some', 21057),
 ('like', 20964),
 ('really', 19695),
 ('It', 19513),
 ('did', 18661),
 ('more', 18338),
 ('all', 18010),
 ('one', 17615),
 ('use', 17487),
 ('they', 17014),
 ('would', 16822),
 ('little', 16798),
 ('sauce', 16769)

#### Вариант предложенный в задании

In [22]:
%%time

sorted(get_word_reviews_count(reviews).items(), key=lambda x: x[1], reverse=True)

Wall time: 19.6 s


[('', 330966),
 ('I', 288243),
 ('the', 266530),
 ('and', 219215),
 ('a', 166467),
 ('it', 133219),
 ('to', 132170),
 ('for', 123162),
 ('of', 109239),
 ('this', 93873),
 ('was', 89617),
 ('recipe', 71751),
 ('in', 62796),
 ('with', 59275),
 ('is', 56519),
 ('so', 46848),
 ('used', 45538),
 ('my', 44644),
 ('but', 43035),
 ('This', 39555),
 ('made', 39363),
 ('that', 38949),
 ('make', 37321),
 ('on', 36121),
 ('as', 35863),
 ('have', 35532),
 ('Thanks', 32325),
 ('good', 31181),
 ('you', 28143),
 ('had', 28047),
 ('great', 27326),
 ('very', 26040),
 ('The', 25845),
 ('them', 25768),
 ('out', 25697),
 ('be', 25514),
 ('just', 25271),
 ('time', 24744),
 ('will', 24097),
 ('not', 23840),
 ('added', 22099),
 ('easy', 21683),
 ('were', 21189),
 ('again', 21144),
 ('some', 21057),
 ('like', 20964),
 ('really', 19695),
 ('It', 19513),
 ('did', 18661),
 ('more', 18338),
 ('all', 18010),
 ('one', 17615),
 ('use', 17487),
 ('they', 17014),
 ('would', 16822),
 ('little', 16798),
 ('sauce', 16769)

## Numba

В файле `rating_predictions.json` хранятся данные о рейтингах рецептов и прогнозных значениях рейтингов для этого рецепта, полученных при помощи модели машинного обучения. 

Напишите несколько версий функции (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от прогнозного значения рейтинга для данного рецепта. 


Замечание 1: в формуле MAPE под $A_t$ понимается рейтинг из отзыва $t$, под $F_t$ - прогнозное значения рейтинга отзыва $t$.

Замечание 2: в результате работы функций должно получиться одно число - MAPE для всего набора данных.

3\.1 Создайте два списка `A_list` и `F_list` на основе файла `rating_predictions.json`. Напишите функцию `mape_lists` без использования векторизованных операций и методов массивов `numpy` и без использования `numba` (проитерируйтесь по спискам и вычислите суммарное значение MAPE для всех элементов, а потом усредните результат).

Измерьте время выполнения данной функции на входных данных `A_list` и `F_list`. Временем, затрачиваемым на создание списков, можно пренебречь.
    

In [23]:
with open(
    "rating_predictions.json",
    "r",
    encoding="utf-8"
) as fp:
    lst_ratings = json.load(fp)
    A_list = []
    F_list = []
    for ratings in lst_ratings:
        A_list.append(ratings['rating'])
        F_list.append(ratings['prediction'])

In [24]:
def mape_lists(lstA, lstF):
    sum_mape = 0
    for i in range(len(lstA)):
        sum_mape += abs((lstA[i] - lstF[i]) / lstA[i])
    res_mape = ((sum_mape)/len(lstA)) * 100
    return res_mape

In [25]:
%%time

mape_lists(A_list, F_list)

Wall time: 21 ms


13.325265503992636

3\.2. Создайте массивы `numpy` `A_array` и `F_array` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numpy` с использованием векторизованных операций и методов массивов `numpy`.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`. Временем, затрачиваемым на создание массивов, можно пренебречь.

In [26]:
A_array, F_array = np.array(A_list), np.array(F_list)

In [27]:
def mape_numpy(A_array, F_array): 
    return np.mean(np.abs((A_array - F_array) / A_array)) * 100

In [28]:
%%time

mape_numpy(A_array, F_array)

Wall time: 1 ms


13.32526550399145

3\.3. Создайте объекты `numba.typed.List` `A_typed` и `F_typed` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numba` без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`. 

Измерьте время выполнения данной функции на входных данных `A_typed` и `F_typed`. Временем, затрачиваемым на создание объектов `numba.typed.List`, можно пренебречь.

Измерьте время выполнения данной функции на входных данных `A_numpy` и `F_numpy`.

In [29]:
A_typed = numba.typed.List(A_list)
F_typed = numba.typed.List(F_list)

In [30]:
@njit
def mape_numba(A_list, F_list):
    sum_mape = 0
    for i in prange(len(A_list)):
        sum_mape += abs((A_list[i] - F_list[i]) / A_list[i])
    res_mape = (sum_mape / len(A_list)) * 100
    return res_mape

In [33]:
%%time

mape_numba(A_typed, F_typed)

Wall time: 0 ns


13.325265503992636

## Векторизация

Сайт-агрегатор устроил акцию: он дарит купоны на посещение ресторана тем пользователям, оставившим отзывы, идентификатор которых является _красивым числом_. Натуральное число называется _красивым_, если первая цифра числа совпадает с последней цифрой числа. 



4\.1 Напишите функцию `is_pretty`, которая для каждого идентификатора пользователя из файла определяет, получит ли он подарок. Запрещается преобразовывать идентификатор пользователя к строке. Подтвердите корректность реализации, продемонстрировав примеры.

In [34]:
ids = reviews["recipe_id"].values
ids

array([ 57993, 142201, 252013, ..., 222001, 354979, 415599], dtype=int64)

In [35]:
def is_pretty(n: int) -> bool:
    first_digit = n // (10**math.floor(math.log10(n)) + 1)
    last_digit = n % 10
    if first_digit == last_digit:
        return True
    else:
        return False

In [36]:
is_pretty_bool_list = list(map(is_pretty, ids))
is_pretty_bool_list

[False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [37]:
is_pretty(57993)

False

In [38]:
is_pretty(142201)

True

4\.2 Посчитайте с помощью функции `is_pretty` количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.

In [39]:
%%time

list(map(is_pretty, ids)).count(True)

Wall time: 131 ms


11958

In [40]:
len(ids[list(map(is_pretty, ids))])

11958

4\.3. При помощи `numpy` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [41]:
np_vect_func = np.vectorize(is_pretty)
np_vect_func(ids)

array([False,  True, False, ..., False, False, False])

In [42]:
%%time

np.count_nonzero(np_vect_func(ids))

Wall time: 78 ms


11958

4\.4. При помощи `numba` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [43]:
@numba.vectorize
def is_pretty_numba(n: int) -> bool:
    first_digit = n // (10**math.floor(math.log10(n)) + 1)
    last_digit = n % 10
    if first_digit == last_digit:
        return 1
    else:
        return 0

In [45]:
%%time

is_pretty_numba(ids).sum()

Wall time: 2 ms


11958